In [1]:
import pandas as pd
import dask.dataframe as dd
import os
import numpy as np

dir = {Your Path to the files}

## Books

In [3]:
books = dd.read_csv(os.path.join(dir, 'books.csv'), dtype={'isbn': 'object',
       'isbn13': 'object',
       'book_id': 'object'})

In [12]:
books.dtypes

isbn                  object
country_code          object
language_code         object
description           object
format                object
publisher             object
num_pages            float64
publication_day      float64
isbn13                object
publication_month    float64
publication_year     float64
image_url             object
book_id               object
title                 object
dtype: object

In [10]:
fields = ['isbn', 'country_code', 'language_code', 'description', 'format', 'publisher', 'isbn13', 'image_url', 'book_id', 'title']

In [11]:
max_dict = {}
for item in fields:
    max_dict[item] = books[item].str.len().max().compute()

In [12]:
max_dict

{'isbn': 10.0,
 'country_code': 2.0,
 'language_code': 9.0,
 'description': 64752.0,
 'format': 110.0,
 'publisher': 241.0,
 'isbn13': 19.0,
 'image_url': 88.0,
 'book_id': 8,
 'title': 255.0}

In [5]:
books[books['description'].str.len() > 60000].compute()

,isbn,country_code,language_code,description,format,publisher,num_pages,publication_day,isbn13,publication_month,publication_year,image_url,book_id,title
20345,NaN,US,eng,BIPOLAR DISORDER:\nA Personal Analysis of My C...,ebook,NaN,125.0,NaN,NaN,NaN,NaN,https://s.gr-assets.com/assets/nophoto/book/11...,7851011,BIPOLAR DISORDER: A Personal Analysis of My Ch...


In [6]:
len(books)

2360655

In [16]:
books.head(2)['description']

0                                                  NaN
1    Anita Diamants international bestseller The Re...
2    Omnibus book club edition containing the Ladie...
3    Addie Downs and Valerie Adler were eight when ...
4                                                  NaN
Name: description, dtype: object

In [17]:
for index, val in books.head()['description'].iteritems():
    print(val)

nan
Anita Diamants international bestseller The Red Tent brilliantly re-created the ancient world of womanhood. Diamant brings her remarkable storytelling skills to Good Harbor -- offering insight to the precarious balance of marriage and career, motherhood and friendship in the world of modern women. The seaside town of Gloucester, Massachusetts is a place where the smell of the ocean lingers in the air and the rocky coast glistens in the Atlantic sunshine. When longtime Gloucester-resident Kathleen Levine is diagnosed with breast cancer, her life is thrown into turmoil. Frightened and burdened by secrets, she meets Joyce Tabachnik -- a freelance writer with literary aspirations -- and a once-in-a-lifetime friendship is born. Joyce has just bought a small house in Gloucester, where she hopes to write as well as vacation with her family. Like Kathleen, Joyce is at a fragile place in her life.A mutual love for books, humor, and the beauty of the natural world brings the two women togeth

In [37]:
books.dropna(subset='title')[books['title'].dropna().str.contains('Karen Blixen', case=False)].compute()

## Authors

In [57]:
authors = dd.read_csv(os.path.join(dir, 'authors.csv'), dtype={'author_id': 'object'})

In [39]:
authors

,author_id,name
npartitions=1,,
,object,object
,...,...


In [40]:
authors.head()

,author_id,name
0,604031,Ronald J. Fields
1,626222,Anita Diamant
2,10333,Barbara Hambly
3,9212,Jennifer Weiner
4,149918,Nigel Pennick


In [20]:
fields = ['author_id', 'name']
max_dict = {}
for item in fields:
    max_dict[item] = authors[item].str.len().max().compute()

In [21]:
max_dict

{'author_id': 8, 'name': 116.0}

In [21]:
authors.isna().sum().compute()

author_id    0
name         5
dtype: int64

In [40]:
authors['author_id'].compute().duplicated().any()

False

## Writes

In [49]:
writes = dd.read_csv(os.path.join(dir, 'writes.csv'), dtype={'book_id': 'object', 'author_id': 'object', 'role': object})

In [26]:
writes.dtypes

book_id      object
author_id    object
role         object
dtype: object

In [28]:
fields = ['book_id', 'author_id', 'role']
max_dict = {}
for item in fields:
    max_dict[item] = writes[item].str.len().max().compute()

In [29]:
max_dict

{'book_id': 8, 'author_id': 8.0, 'role': 36.0}

In [19]:
writes.isna().sum().compute()

book_id            0
author_id        537
role         2689029
dtype: int64

In [24]:
writes_no_auth = writes[writes['author_id'].isna()].compute()

In [25]:
writes_no_auth

,book_id,author_id,role
2400,23699819,NaN,NaN
15165,2597774,NaN,NaN
20221,18521522,NaN,NaN
25353,28253116,NaN,NaN
30671,17796597,NaN,NaN
...,...,...,...
3292670,26206589,NaN,NaN
3293465,1944688,NaN,NaN
3304936,26253723,NaN,NaN
3309356,23777395,NaN,NaN


In [43]:
writes_df = writes.compute()

In [44]:
writes_df = writes_df.dropna(subset='author_id')

In [45]:
writes_df.isna().sum()

book_id            0
author_id          0
role         2688492
dtype: int64

In [52]:
group = writes_df[writes_df[['book_id', 'author_id']].duplicated()].groupby(by=['book_id', 'author_id'])

In [54]:
count = 0
for name, row in group:
    if count < 5:
        print(name)
        print(group)
        count += 1
    else:
        break

('10004537', '5740762')
('10005636', '5736191')
('1001131', '390346')
('1001229', '5690319')
('1001379', '5791535')


In [56]:
writes_df[writes_df['book_id'] == '10004537']

,book_id,author_id,role
506939,10004537,5740762,NaN
506940,10004537,5740762,NaN


In [60]:
writes_df.duplicated().sum()

4853

In [63]:
writes_df = writes_df.drop_duplicates()

In [66]:
writes_df.to_csv(os.path.join(dir, 'writes.csv'), index=0)

In [67]:
writes_check = pd.read_csv(os.path.join(dir, 'writes.csv'))

In [68]:
writes_check

,book_id,author_id,role
0,5333265,604031,NaN
1,1333909,626222,NaN
2,7327624,10333,NaN
3,6066819,9212,NaN
4,287140,149918,NaN
...,...,...,...
3318763,26168430,2448,NaN
3318764,26168430,3460250,NaN
3318765,2342551,82312,Editor
3318766,22017381,7789809,NaN


## Genres

In [63]:
genres = dd.read_csv(os.path.join(dir, 'genres.csv'), dtype={'book_id': 'object', 'genre': 'object'})

In [42]:
genres.head()

,book_id,genre,num_tagged
0,5333265,"history, historical fiction, biography",1.0
1,1333909,"history, historical fiction, biography",5.0
2,287141,"history, historical fiction, biography",9.0
3,287149,"history, historical fiction, biography",1.0
4,6066814,"history, historical fiction, biography",38.0


In [35]:
fields = ['book_id', 'genre']
max_dict = {}
for item in fields:
    max_dict[item] = genres[item].str.len().max().compute()

In [36]:
max_dict

{'book_id': 8, 'genre': 38}

In [17]:
genres.isna().sum().compute()

book_id       0
genre         0
num_tagged    0
dtype: int64

In [42]:
genres[['book_id', 'genre']].compute().duplicated().any()

False

## Similar

In [33]:
similar = dd.read_csv(os.path.join(dir, 'similar.csv'), dtype={'book_id': 'object', 'similar_books': 'object'})

In [43]:
similar.dtypes

book_id          object
similar_books    object
dtype: object

In [40]:
fields = ['book_id', 'similar_books']
max_dict = {}
for item in fields:
    max_dict[item] = similar[item].str.len().max().compute()

In [41]:
max_dict

{'book_id': 8, 'similar_books': 8.0}

In [14]:
similar.isna().sum().compute()

book_id                0
similar_books    1262038
dtype: int64

In [29]:
similar_df = similar.compute()

In [30]:
similar_df

,book_id,similar_books
0,5333265,NaN
1,1333909,8709549
2,1333909,17074050
3,1333909,28937
4,1333909,158816
...,...,...
4518897,11419866,1835589
4518898,11419866,1739458
4518899,11419866,1534367
4518900,11419866,8662586


In [31]:
similar_df = similar_df.dropna()

In [32]:
similar_df

,book_id,similar_books
1,1333909,8709549
2,1333909,17074050
3,1333909,28937
4,1333909,158816
5,1333909,228563
...,...,...
4518897,11419866,1835589
4518898,11419866,1739458
4518899,11419866,1534367
4518900,11419866,8662586


In [35]:
similar_df.duplicated().any()

False

In [36]:
similar_df.to_csv(os.path.join(dir, 'similar.csv'), index=0)

In [37]:
sim_check = pd.read_csv(os.path.join(dir, 'similar.csv'))

In [38]:
sim_check

,book_id,similar_books
0,1333909,8709549
1,1333909,17074050
2,1333909,28937
3,1333909,158816
4,1333909,228563
...,...,...
16809712,11419866,1835589
16809713,11419866,1739458
16809714,11419866,1534367
16809715,11419866,8662586


## Interactions

In [68]:
ints = dd.read_csv(os.path.join(dir, 'interactions.csv'), dtype={'user_id': 'object', 'book_id': 'object', 'is_read': 'object', 'rating': 'object', 'is_reviewed': 'object'})

In [44]:
ints.head()

,Unnamed: 0,user_id,is_read,rating,is_reviewed,book_id
0,0,0,1,5,0,12
1,1,201,1,3,0,12
2,2,339,1,3,1,12
3,3,683,1,5,0,12
4,4,1856,0,0,0,12


In [45]:
len(ints)

228648342

In [9]:
ints.dtypes

Unnamed: 0      int64
user_id        object
is_read        object
rating         object
is_reviewed    object
book_id        object
dtype: object

In [10]:
fields = ['user_id', 'book_id', 'is_read', 'rating', 'is_reviewed']
max_dict = {}
for item in fields:
    max_dict[item] = ints[item].str.len().max().compute()

In [11]:
max_dict

{'user_id': 6, 'book_id': 8, 'is_read': 1, 'rating': 1, 'is_reviewed': 1}

In [12]:
ints.isna().sum().compute()

Unnamed: 0     0
user_id        0
is_read        0
rating         0
is_reviewed    0
book_id        0
dtype: int64

target_dir = {Your target directory}

In [47]:
ints

,Unnamed: 0,user_id,is_read,rating,is_reviewed,book_id
npartitions=104,,,,,,
,int64,object,object,object,object,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [ ]:
ints.to_csv(os.path.join(target_dir, 'interactions.csv'),index=False, single_file=True)

In [50]:
intt = dd.read_csv((os.path.join(target_dir, 'interactions.csv')), dtype={'user_id': 'object', 'book_id': 'object', 'is_read': 'object', 'rating': 'object', 'is_reviewed': 'object'})

In [51]:
intt.head()

,Unnamed: 0,user_id,is_read,rating,is_reviewed,book_id
0,0,0,1,5,0,12
1,1,201,1,3,0,12
2,2,339,1,3,1,12
3,3,683,1,5,0,12
4,4,1856,0,0,0,12


In [53]:
import csv

In [54]:
with open(os.path.join(dir, 'interactions.csv')) as f:
    reader = csv.reader(f)
    row1 = next(reader)  # gets the first line
    print(row1)

['', 'user_id', 'is_read', 'rating', 'is_reviewed', 'book_id']


---

In [23]:
books_s = books.sample(frac=.0001)

In [24]:
books_s = books_s.compute()

In [25]:
books_s

,isbn,country_code,language_code,description,format,publisher,num_pages,publication_day,isbn13,publication_month,publication_year,image_url,book_id,title
40697,NaN,US,por,A ideia para este livro surgiu com o filme Sex...,Hardcover,Bertrand Editora,159.0,NaN,9789722518680,1.0,2009.0,https://images.gr-assets.com/books/1314909556m...,9223578,Cartas de Amor de Grandes Homens
1363,1594488452,US,NaN,The enchanting story of a midwestern girl who ...,NaN,NaN,320.0,NaN,9781594488450,NaN,NaN,https://images.gr-assets.com/books/1337466907m...,13588166,Laura Lamont's Life in Pictures
67088,0060125489,US,eng,"NO PARACHUTE is an exciting find, a uniquely a...",Mass Market Paperback,Pocket Books,234.0,NaN,9780060125486,10.0,1971.0,https://s.gr-assets.com/assets/nophoto/book/11...,2668104,No Parachute: A Fighter Pilot In World War I
13756,0099421860,US,NaN,NaN,Paperback,Vintage,272.0,5.0,9780099421863,9.0,2002.0,https://s.gr-assets.com/assets/nophoto/book/11...,4836,Fury
2290,NaN,US,eng,"ANNOTATED:* Contains literary critiques, detai...",NaN,NaN,NaN,NaN,2940012252319,NaN,NaN,https://s.gr-assets.com/assets/nophoto/book/11...,10846266,The Complete Speeches of Mark Twain/The Burles...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45615,0140623868,US,eng,The Turn of the Screw is a subtle and unconven...,Paperback,Penguin Books,320.0,NaN,9780140623864,NaN,1994.0,https://images.gr-assets.com/books/1390100762m...,7723793,The Turn of the Screw
61291,0985159022,US,eng,When control of the Vatican is at stake--money...,Paperback,Lyons McNamara LLC,324.0,22.0,9780985159023,1.0,2013.0,https://images.gr-assets.com/books/1361583436m...,17279868,Archangels: Rise of the Jesuits
63220,NaN,US,NaN,The quest for answers--and ultimate survival--...,Kindle Edition,Thomas Nelson,400.0,3.0,NaN,9.0,2013.0,https://images.gr-assets.com/books/1371849073m...,18104033,The Shadow Lamp (Bright Empires #4)
46148,0717283364,US,eng,Relive Walt Disneys 19th full-length animated ...,Hardcover,Grolier Books,44.0,NaN,9780717283361,NaN,1993.0,https://s.gr-assets.com/assets/nophoto/book/11...,1350312,The Jungle Book


In [26]:
books_s.to_csv(os.path.join(dir, 'test_books.csv'), index=False)

In [36]:
books_s = pd.read_csv(os.path.join(dir, 'test_books.csv'), dtype={'book_id': 'object'})

In [37]:
books_s.dtypes

isbn                  object
country_code          object
language_code         object
description           object
format                object
publisher             object
num_pages            float64
publication_day      float64
isbn13               float64
publication_month    float64
publication_year     float64
image_url             object
book_id               object
title                 object
dtype: object

In [29]:
book_list = books_s['book_id']

In [42]:
similar_s = similar.merge(books_s[['book_id']], on='book_id')

In [43]:
similar_s = similar_s.compute()

In [44]:
similar_s

,book_id,similar_books
0,13588166,15960120
1,13588166,13056378
2,13588166,17433971
3,13588166,13259954
4,13588166,25431247
...,...,...
528,18104033,12891597
529,18104033,12079838
530,18104033,17264330
531,18104033,16159644


In [47]:
similar_s['similar_books'].isin(list(similar_s['book_id'])).any()

False

In [48]:
similar_s.to_csv(os.path.join(dir, 'test_similar.csv'), index=False)

In [51]:
writes_s = writes.merge(books_s[['book_id']], on='book_id')

In [55]:
writes_s = writes_s.compute()

In [56]:
writes_s.to_csv(os.path.join(dir, 'test_writes.csv'), index=False)

In [58]:
authors_s = authors.merge(writes_s[['author_id']], on='author_id')

In [60]:
authors_s = authors_s.compute()

In [61]:
authors_s

,author_id,name
0,14425920,Chance Carter
1,2737,Max Lucado
2,7084,Karl Marx
3,4128,J.M. Coetzee
4,1244,Mark Twain
...,...,...
334,3889699,Donna Gummelt
335,3889700,Dondino Melchiorre
336,3265024,Randy Hugh Wall
337,3889701,Juan D. Varela


In [62]:
authors_s.to_csv(os.path.join(dir, 'test_authors.csv'), index=False)

In [64]:
genres_s = genres.merge(books_s[['book_id']], on='book_id')

In [65]:
genres_s = genres_s.compute()

In [66]:
genres_s

,book_id,genre,num_tagged
0,13588166,"history, historical fiction, biography",112.0
1,13588166,fiction,271.0
2,10846266,"history, historical fiction, biography",2.0
3,33448958,"history, historical fiction, biography",34.0
4,33448958,fiction,2100.0
...,...,...,...
310,6016824,young-adult,1.0
311,25234837,young-adult,1.0
312,2868526,young-adult,-1.0
313,79491,"comics, graphic",49.0


In [67]:
genres_s.to_csv(os.path.join(dir, 'test_genres.csv'), index=False)

In [69]:
ints_s = ints.merge(books_s[['book_id']], on='book_id')

In [70]:
ints_s = ints_s.compute()

In [82]:
ints_s = ints_s.drop('Unnamed: 0', axis=1)

In [83]:
ints_s.columns

Index(['user_id', 'is_read', 'rating', 'is_reviewed', 'book_id'], dtype='object')

In [81]:
ints_s.to_csv(os.path.join(dir, 'test_interactions.csv'), index=False)